### Scorecard Calibration

Point-in-Time (PiT) calibration is a method used to adjust the predicted probabilities of a model so that they align with the actual observed default rates. If the observed default rate at the time of calibration is the same as it was when the scorecard was built, then the calibration process would not lead to any changes in the predicted probabilities, and hence, no improvement in the model's performance as measured by the confusion matrix or other metrics.

The purpose of calibration is to ensure that the model's predicted probabilities accurately reflect the actual likelihood of the event (in this case, default). If the underlying default rate hasn't changed since the model was built, then the model's predictions are already well-calibrated, and further calibration isn't necessary.

In [ ]:
def create_scorecard(model, X, target_score, target_odds, pdo):
    # Compute probability of default
    X_copy = X.copy()
    X_copy["probabilities"] = model.predict(X_copy)

    # Compute scores
    factor = pdo / np.log(2)
    offset = target_score - (factor * np.log(target_odds))

    X_copy["score"] = offset + factor * np.log(
        X_copy["probabilities"] / (1 - X_copy["probabilities"])
    )
    return X_copy

target_score = 600
target_odds = 50
pdo = 20
X_train_scores = create_scorecard(model_fit_glm, X_train, target_score, target_odds, pdo)
X_train_scores = X_train_scores['score']

In [ ]:
# Define the model
X_train_scores = sm.add_constant(X_train_scores)
calibrated_model = sm.GLM(y_train, X_train_scores, family=sm.families.Binomial())

# Fit the model
calibrated_model_fit_glm = calibrated_model.fit()

# Print out the statistics
print(calibrated_model_fit_glm.summary())

In [ ]:
# Create df_calibrated_train and df_calibrated_test
df_calibrated_train = pd.concat([X_train_scores, y_train], axis=1)

# Update test data 
X_test_scores = create_scorecard(model_fit_glm, X_test, target_score, target_odds, pdo)
X_test_scores = X_test_scores['score']
X_test_scores = sm.add_constant(X_test_scores)
df_calibrated_test = pd.concat([X_test_scores, y_test], axis=1)

In [ ]:
# Create VM dataset
vm_train_calibrated_ds = vm.init_dataset(dataset=df_calibrated_train,
                        target_column=target_column)
vm_test_calibrated_ds = vm.init_dataset(dataset=df_calibrated_test,
                        target_column=target_column)

# Create VM model
vm_model_glm_calibrated = vm.init_model(
    model = calibrated_model_fit_glm, 
    train_ds=vm_train_calibrated_ds, 
    test_ds=vm_test_calibrated_ds)

In [ ]:
from validmind.tests.model_validation.statsmodels.LogRegressionConfusionMatrix import LogRegressionConfusionMatrix

test_context = TestContext(model= vm_model_glm_calibrated)

# Configure test parameters
params = {
    "cut_off_threshold": 0.5,
}

metric = LogRegressionConfusionMatrix(test_context, params)
metric.run()
await metric.result.log()
metric.result.show()

### Probability of Default by Rating Class

In [ ]:
from validmind.tests.model_validation.statsmodels.PDRatingClassPlot import PDRatingClassPlot

# Configure test parameters
params = {
    "rating_classes": ['A','B','C','D'],
    "title": "PD by Rating Class",
}
test_context = TestContext(model= vm_model_glm_calibrated)
metric = PDRatingClassPlot(test_context, params)
metric.run()
await metric.result.log()
metric.result.show()